# HyperDrive

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project", subscription_id='8bad5f66-e234-46b1-b07f-d3cfd6255f67')
exp = Experiment(workspace=ws, name="udacity-project-kenny")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacity-project
Azure region: eastus
Subscription id: 8bad5f66-e234-46b1-b07f-d3cfd6255f67
Resource group: udacity-project


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

cluster_name = "udacity-project"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("I found an exisiting cluster so I'm using it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           vm_priority = 'lowpriority',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

I found an exisiting cluster so I'm using it.


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
params = {
    "C": uniform(0.0, 1.0),
    "max_iter": choice(50, 100, 150, 200, 250, 300)
}
ps = RandomParameterSampling(params)

# Specify a Policy
policy = BanditPolicy(
    slack_factor=0.1,
    evaluation_interval=1
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory=".",
    compute_target=compute_target,
    entry_script="train.py"
)

primary_metric_goal = PrimaryMetricGoal("MAXIMIZE")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=primary_metric_goal,
    max_concurrent_runs=4,
    max_total_runs=40
)

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [14]:
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
params = best_run.get_details()['runDefinition']['arguments']

print('Id of the best run: ', best_run.id)
print('Accuracy of the best run:', best_run_metrics['Accuracy'])
print('C:', params[1])
print('max_iter:', params[3])

best_run.download_file("./outputs/model.joblib", "logreg_best_model.joblib")

Id of the best run:  HD_b81e5cbe-63ff-474d-999e-cc9e1f95bb75_12
Accuracy of the best run: 0.9167273610099539
C: 0.25137065561901684
max_iter: 200


In [16]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

